<h1> Opening a New Indian Restaurant in Toronto, Canada

<h5>
    <i><h3>Contents </i>  <br/> 
1.Build a dataframe of neighborhoods in Toronto,Canada by web scraping the data from Wikipedia page <br />
2.Get the geographical coordinates of the neighborhoods<br />
3.Obtain the venue data for the neighborhoods from Foursquare API<br />
4.Explore and cluster the neighborhoods<br />
5.Select the best cluster to open a new Indian Restaurant<br />

<h2> 1.Importing Required Libraries

In [2]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
! pip install --user bs4
! pip install --user parse
! pip install --user lxml
! pip install --user geocoder
!pip install --user geopy
from  geopy.geocoders import Nominatim
import parse
import matplotlib.pyplot as plt
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
import requests
from requests import get
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
%matplotlib inline

<h2>2.Getting Required data of Neighbourhoods

In [64]:
#getting table from url
url = "https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [65]:
demo_toronto= html_soup.find_all('table',class_="wikitable sortable")

In [66]:
for a in html_soup.findAll('a'):
    a.replaceWithChildren()

In [67]:
#reading html file into list
toronto_table=pd.read_html(str(demo_toronto))

In [68]:
#converting into dataframe
toronto_table=toronto_table[0]

In [69]:
toronto_table=toronto_table.iloc[1:,:]

In [70]:
#selecting required columns
toronto_table=toronto_table.iloc[:,[0,10,11]]
toronto_table.columns

Index(['Name', 'Second most common language (after English) by name',
       'Second most common language (after English) by percentage'],
      dtype='object')

In [72]:
toronto_table.columns[[1,2]]

Index(['Second most common language (after English) by name', 'Second most common language (after English) by percentage'], dtype='object')

In [73]:
toronto_table.rename(columns={'Second most common language (after English) by name':'Name_lang','Second most common language (after English) by percentage':'Percentage'},inplace=True)

In [74]:
toronto_table

,Name,Name_lang,Percentage
1,Agincourt,Cantonese (19.3%),19.3% Cantonese
2,Alderwood,Polish (6.2%),06.2% Polish
3,Alexandra Park,Cantonese (17.9%),17.9% Cantonese
4,Allenby,Russian (1.4%),01.4% Russian
5,Amesbury,Spanish (6.1%),06.1% Spanish
...,...,...,...
170,Woburn,Gujarati (9.1%),09.1% Gujarati
171,Wychwood,Portuguese (2.7%),02.7% Portuguese
172,York Mills,Korean (4.0%),04.0% Korean
173,York University Heights,Italian (6.6%),06.6% Italian


In [75]:
#spliting the percentage columns to get name of language and % as separate thing
df = pd.DataFrame(toronto_table.Percentage.str.split(' ',1).tolist(),
                                   columns = ['Percentage','Name_lang'])

In [76]:
df

,Percentage,Name_lang
0,19.3%,Cantonese
1,06.2%,Polish
2,17.9%,Cantonese
3,01.4%,Russian
4,06.1%,Spanish
...,...,...
169,09.1%,Gujarati
170,02.7%,Portuguese
171,04.0%,Korean
172,06.6%,Italian


In [77]:
toronto_table=toronto_table.reset_index(drop=True)

In [78]:
#updating the toronto table
toronto_table.update(df)

In [79]:
toronto_table['Percentage']=toronto_table['Percentage'].str.replace('%','')

In [80]:
#converting the percentage into float
toronto_table['Percentage']=toronto_table['Percentage'].astype('float')

In [81]:
toronto_table

,Name,Name_lang,Percentage
0,Agincourt,Cantonese,19.3
1,Alderwood,Polish,6.2
2,Alexandra Park,Cantonese,17.9
3,Allenby,Russian,1.4
4,Amesbury,Spanish,6.1
...,...,...,...
169,Woburn,Gujarati,9.1
170,Wychwood,Portuguese,2.7
171,York Mills,Korean,4.0
172,York University Heights,Italian,6.6


In [82]:
#Selecting Neighbourhoods having Indian Languages as Second most Dominating
Lang=['Assamese', 'Bengali', 'Gujarati', 'Hindi', 'Kannada', 'Kashmiri', 'Konkani', 'Malayalam', 'Marathi', 'Nepali', 'Oriya', 'Punjabi', 'Sanskrit', 'Sindhi', 'Tamil', 'Telugu', 'Urdu', 'Bodo', 'Santhali', 'Maithili', 'Dogri', 'Manipuri']
toronto_indian=toronto_table[toronto_table['Name_lang'].isin(Lang)]

In [83]:
#dataframe containing names of neighbourhoods with majority of indian languages speaking people and their percentage
toronto_indian

,Name,Name_lang,Percentage
12,Bendale,Tamil,3.7
27,Clairville,Punjabi,12.0
29,Cliffcrest,Tamil,1.5
32,Crescent Town,Bengali,18.1
41,Dorset Park,Tamil,9.5
48,Eglinton East,Tamil,9.7
54,Flemingdon Park,Urdu,6.3
56,Fort York/Liberty Village,Hindi,1.7
68,Highland Creek,Tamil,5.1
76,Humbermede,Punjabi,9.7


<h2>3.Getting Location Data of each Neighbourhood from Geocoder

In [84]:
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent='CA')
latitude,longitude=[],[]
for name in toronto_indian['Name']:
    try:
        name=name.split("/")[0]
        print(name)
        # conveneint function to delay between geocoding calls
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        location = geocode("{},Toronto,CA".format(name))
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except AttributeError:
        latitude.append("Not found")
        longitude.append("Not found")
    
    

Bendale
Clairville
Cliffcrest
Crescent Town
Dorset Park
Eglinton East
Flemingdon Park
Fort York
Highland Creek
Humbermede
Humberwood
Malvern
Morningside
Oakridge
O'Connor–Parkview
Regent Park
Rouge
Scarborough Junction
Scarborough City Centre
Scarborough Village
Smithfield
Thistletown
Thorncliffe Park
Victoria Village
West Hill
Woburn


In [85]:
#latitude and longitudes of neighbourhoods in toronto_indian dataframe
toronto_indian['Latitude']=latitude
toronto_indian["Longitude"]=longitude

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
toronto_indian

,Name,Name_lang,Percentage,Latitude,Longitude
12,Bendale,Tamil,3.7,43.7535,-79.2553
27,Clairville,Punjabi,12.0,Not found,Not found
29,Cliffcrest,Tamil,1.5,43.7219,-79.2362
32,Crescent Town,Bengali,18.1,43.6954,-79.2931
41,Dorset Park,Tamil,9.5,43.7528,-79.2821
48,Eglinton East,Tamil,9.7,43.7396,-79.2323
54,Flemingdon Park,Urdu,6.3,43.7184,-79.3332
56,Fort York/Liberty Village,Hindi,1.7,43.6387,-79.4035
68,Highland Creek,Tamil,5.1,43.7901,-79.1733
76,Humbermede,Punjabi,9.7,Not found,Not found


In [87]:
toronto_indian=toronto_indian.reset_index(drop=True)

In [88]:
#removing rows which doesn't have latitudes and longitudes of neighbourhoods
toronto_indian=toronto_indian[toronto_indian['Latitude']!='Not found']

In [89]:
#dataframe containing locations of neighbourhoods
toronto_indian

,Name,Name_lang,Percentage,Latitude,Longitude
0,Bendale,Tamil,3.7,43.7535,-79.2553
2,Cliffcrest,Tamil,1.5,43.7219,-79.2362
3,Crescent Town,Bengali,18.1,43.6954,-79.2931
4,Dorset Park,Tamil,9.5,43.7528,-79.2821
5,Eglinton East,Tamil,9.7,43.7396,-79.2323
6,Flemingdon Park,Urdu,6.3,43.7184,-79.3332
7,Fort York/Liberty Village,Hindi,1.7,43.6387,-79.4035
8,Highland Creek,Tamil,5.1,43.7901,-79.1733
10,Humberwood,Punjabi,26.7,43.7225,-79.546
11,Malvern,Tamil,12.2,43.8092,-79.2217


In [90]:
#visuaizing neighbourhoods
venues_map = folium.Map(location=[43.7535,-79.2553], zoom_start=13) 
for lat, lon, poi in zip(toronto_indian['Latitude'], toronto_indian['Longitude'], toronto_indian['Name']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(venues_map)
    
venues_map



<h2>4.Using Foursquare API to explore the restaurants i.e.Indian,Asian,Fast food nearby our neighbourhoods  

In [91]:
CLIENT_ID = 'your id' #  Foursquare ID
CLIENT_SECRET = 'your secret' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [92]:
import requests
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [93]:
toronto_venues = getNearbyVenues(names=toronto_indian['Name'],
                                   latitudes=toronto_indian['Latitude'],
                                   longitudes=toronto_indian['Longitude']
                                  )


Bendale
Cliffcrest
Crescent Town
Dorset Park
Eglinton East
Flemingdon Park
Fort York/Liberty Village
Highland Creek
Humberwood
Malvern
Morningside
Oakridge
O'Connor–Parkview
Regent Park/Trefann Court
Rouge
Scarborough Junction
Scarborough City Centre
Scarborough Village
Smithfield
Thistletown
Thorncliffe Park
Victoria Village
West Hill
Woburn


In [94]:
toronto_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bendale,43.753520,-79.255336,Thomson Memorial Dog Park,43.756253,-79.253294,Dog Run
1,Bendale,43.753520,-79.255336,Top Food Supermarket,43.752396,-79.259946,Grocery Store
2,Bendale,43.753520,-79.255336,Ding Ho Chinese Food,43.753507,-79.253542,Chinese Restaurant
3,Bendale,43.753520,-79.255336,Brimley Rd & Lawrence Ave E,43.753405,-79.255625,Intersection
4,Bendale,43.753520,-79.255336,Thomson Park Tennis Club,43.756074,-79.255153,Tennis Court
...,...,...,...,...,...,...,...
314,Woburn,43.759824,-79.225291,Tim Hortons,43.760028,-79.226545,Coffee Shop
315,Woburn,43.759824,-79.225291,Canadian Tire,43.758451,-79.226523,Hardware Store
316,Woburn,43.759824,-79.225291,KFC,43.757943,-79.229483,Fast Food Restaurant
317,Woburn,43.759824,-79.225291,EB Games,43.759534,-79.228468,Video Game Store


In [95]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighbourhood,Afghan Restaurant,American Restaurant,Animal Shelter,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,Bakery,Bank,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Yoga Studio
0,Bendale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bendale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bendale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bendale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bendale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
316,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
317,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [96]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped=toronto_grouped[['Neighbourhood','Indian Restaurant','Asian Restaurant','Fast Food Restaurant']]
toronto_grouped

,Neighbourhood,Indian Restaurant,Asian Restaurant,Fast Food Restaurant
0,Bendale,0.000000,0.00,0.142857
1,Cliffcrest,0.000000,0.00,0.285714
2,Crescent Town,0.000000,0.00,0.000000
3,Dorset Park,0.100000,0.10,0.100000
4,Eglinton East,0.166667,0.00,0.333333
5,Flemingdon Park,0.000000,0.00,0.076923
6,Fort York/Liberty Village,0.000000,0.00,0.000000
7,Highland Creek,0.000000,0.00,0.000000
8,Humberwood,0.000000,0.00,0.000000
9,Malvern,0.000000,0.00,0.153846


In [97]:
toronto_restro=toronto_grouped

In [98]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [99]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    print(ind)
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_restro['Neighbourhood']

for ind in np.arange(toronto_restro.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_restro.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

0
1
2


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bendale,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
1,Cliffcrest,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
2,Crescent Town,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
3,Dorset Park,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
4,Eglinton East,Fast Food Restaurant,Indian Restaurant,Asian Restaurant


In [100]:
neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bendale,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
1,Cliffcrest,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
2,Crescent Town,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
3,Dorset Park,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
4,Eglinton East,Fast Food Restaurant,Indian Restaurant,Asian Restaurant
5,Flemingdon Park,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
6,Fort York/Liberty Village,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
7,Highland Creek,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
8,Humberwood,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
9,Malvern,Fast Food Restaurant,Asian Restaurant,Indian Restaurant


<h2>4.Clustering the Neighbourhoods

In [101]:
from sklearn.cluster import KMeans
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int32)

In [102]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_indian.rename(columns={'Name':'Neighbourhood'},inplace=True)
toronto_merged = toronto_indian

# merge toronto_grouped with toronto_indian to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged

,Neighbourhood,Name_lang,Percentage,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Bendale,Tamil,3.7,43.7535,-79.2553,0,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
2,Cliffcrest,Tamil,1.5,43.7219,-79.2362,1,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
3,Crescent Town,Bengali,18.1,43.6954,-79.2931,0,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
4,Dorset Park,Tamil,9.5,43.7528,-79.2821,0,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
5,Eglinton East,Tamil,9.7,43.7396,-79.2323,1,Fast Food Restaurant,Indian Restaurant,Asian Restaurant
6,Flemingdon Park,Urdu,6.3,43.7184,-79.3332,0,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
7,Fort York/Liberty Village,Hindi,1.7,43.6387,-79.4035,0,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
8,Highland Creek,Tamil,5.1,43.7901,-79.1733,0,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
10,Humberwood,Punjabi,26.7,43.7225,-79.546,0,Fast Food Restaurant,Asian Restaurant,Indian Restaurant
11,Malvern,Tamil,12.2,43.8092,-79.2217,0,Fast Food Restaurant,Asian Restaurant,Indian Restaurant


<h2>5.Visualizing the Clusters

In [103]:
map_clusters = folium.Map(location=[43.676357,-79.293031], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>6.Examining the Clusters

In [108]:
toronto_merged=toronto_merged.reset_index(drop=True)

In [110]:
toronto_restro[toronto_merged['Cluster Labels']==0]

,Neighbourhood,Indian Restaurant,Asian Restaurant,Fast Food Restaurant
0,Bendale,0.000000,0.00,0.142857
2,Crescent Town,0.000000,0.00,0.000000
3,Dorset Park,0.100000,0.10,0.100000
5,Flemingdon Park,0.000000,0.00,0.076923
6,Fort York/Liberty Village,0.000000,0.00,0.000000
7,Highland Creek,0.000000,0.00,0.000000
8,Humberwood,0.000000,0.00,0.000000
9,Malvern,0.000000,0.00,0.153846
10,Morningside,0.000000,0.00,0.076923
11,O'Connor–Parkview,0.000000,0.00,0.133333


In [111]:
toronto_restro[toronto_merged['Cluster Labels']==1]

,Neighbourhood,Indian Restaurant,Asian Restaurant,Fast Food Restaurant
1,Cliffcrest,0.000000,0.0,0.285714
4,Eglinton East,0.166667,0.0,0.333333
14,Rouge,0.000000,0.0,0.500000
15,Scarborough City Centre,0.000000,0.0,0.000000
23,Woburn,0.000000,0.0,0.210526


In [112]:
toronto_restro[toronto_merged['Cluster Labels']==2]

,Neighbourhood,Indian Restaurant,Asian Restaurant,Fast Food Restaurant
19,Thistletown,0.307692,0.0,0.076923


<h3>Observations</h3><br />

Most of the Restaurants have highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no Indian/Asian Restaurant in the neighborhoods. This represents a great opportunity and high potential areas to open new Indian Restro as there is very little to no competition from existing Restro's. Meanwhile, Indian Restro's in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of Restaurants.Therefore, this project recommends Investors to capitalize on these findings to open new Indian Restaurants in neighborhoods in cluster 0 which have high percentages of Language meter like in Crescent Town, Humberwood etc. Property developers with unique selling propositions to stand out from the competition can also open new Restaurants in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of Indian Restaurant and suffering from intense competition.